In [1]:
%matplotlib inline
import os, sys, inspect, io, pickle
import matplotlib.pyplot as plt
from math import floor
import keypoint_helpers
import state_helpers

cmd_folder = os.path.realpath(
    os.path.dirname(
        os.path.abspath(os.path.split(inspect.getfile( inspect.currentframe() ))[0])))

if cmd_folder not in sys.path:
    sys.path.insert(0, cmd_folder)
    
with open('test.pickle', 'rb') as handle:
    kpts = pickle.load(handle)
    
from transitions import *
from transitions.extensions import GraphMachine
from IPython.display import Image, display, display_png

class Matter(object):
    def is_valid(self):
        return True
    
    def is_not_valid(self):
        return False
    
    def is_also_valid(self):
        return True
    
    # graph object is created by the machine
    def show_graph(self, **kwargs):
        stream = io.BytesIO()
        self.get_graph(**kwargs).draw(stream, prog='dot', format='png')
        display(Image(stream.getvalue()))

ModuleNotFoundError: No module named 'transitions'

In [ ]:
kpts_inverted = state_helpers.invert_keypoints(kpts)

In [ ]:
%%capture
keypoints = keypoint_helpers.get_keypoint_labels()

fig, axes = plt.subplots(9, 2,figsize=(15,30))
flat_axes = [item for sublist in axes for item in sublist]

for i, ax in enumerate(flat_axes):
    ax.grid()
    ax.set_xlim(-1,1)
    ax.set_ylim(-1,1)
    # add 1 to skip second plot
    if i == 0:
         ax.set_title(keypoints[i])
    elif i == 1:
        continue
    else:
        ax.set_title(keypoints[i-1])
        
fig.tight_layout()

In [ ]:
for frame in kpts_inverted:
    for i, point in enumerate(frame):
        # add 1 to skip second plot
        plot_index = i
        if i > 0:
            plot_index+=1
        flat_axes[plot_index].scatter(point[0],point[1])
fig

In [ ]:
%%capture
fig, axes = plt.subplots(9, 2,figsize=(15,30))
flat_axes = [item for sublist in axes for item in sublist]

for i, ax in enumerate(flat_axes):
    ax.grid()
    ax.set_ylim(-.6,.6)
    ax.set_xlabel('Frame (time ~.4 sec)')
    ax.set_ylabel('X measure')
    # add 1 to skip second plot
    if i == 0:
         ax.set_title(keypoints[i] + ' X dim')
    elif i == 1:
        continue
    else:
        ax.set_title(keypoints[i-1] + ' X dim')

fig.tight_layout()

In [ ]:
import copy

kpt_dict_x = {
    'nose': [],
    'left_eye': [],
    'right_eye': [],
    'left_ear': [],
    'right_ear': [],
    'left_shoulder': [],
    'right_shoulder': [],
    'left_elbow': [],
    'right_elbow': [],
    'left_wrist': [],
    'right_wrist': [],
    'left_hip': [],
    'right_hip': [],
    'left_knee': [],
    'right_knee': [],
    'left_ankle': [],
    'right_ankle': []
}

kpt_dict_y = copy.deepcopy(kpt_dict_x)

time = list(range(len(kpts_inverted)))

for frame in kpts_inverted:
    for i, point in enumerate(frame):
        kpt_dict_x[keypoints[i]].append(point[0])
        kpt_dict_y[keypoints[i]].append(point[1])

In [ ]:
for i, kpt in enumerate(keypoints):
    if i < 1:
        flat_axes[i].plot(time, kpt_dict_x[kpt],'.r-')
    elif i > 1:
        flat_axes[i+1].plot(time, kpt_dict_x[kpt],'.r-')
    
fig

In [ ]:
import numpy as np

Fs = 2  # sampling rate
Ts = 1.0/Fs # sampling interval


t = np.arange(0,len(kpts_inverted),Ts) # time vector
y = kpt_dict_x['nose']

n = len(y) # length of the signal
window = np.hamming(n)

k = np.arange(n)
T = n/Fs
frq = k/T # two sides frequency range
frq = frq[range(n//2)] # one side frequency range

Y = np.fft.fft(y*window)/n # fft computing and normalization
Y = Y[range(n//2)]

plt.plot(frq,abs(Y),'r') # plotting the spectrum
plt.xlabel('Freq (Hz)')
plt.ylabel('|Y(freq)|')